# Versatile Async API Notebook

This notebook replaces the interactive `start.py` flow with a reusable async workflow.

## Goals
- Keep API context configurable in `.env` (`env_sample` template).
- Use one generic async client for different APIs.
- Run both in local Jupyter and Google Colab.

## Before running
1. Copy `env_sample` to `.env`.
2. Fill your token and endpoint mappings.

In [ ]:
from utils.downloader_api import ensure_runtime

active_dir = ensure_runtime()
print(f"Running from: {active_dir}")

In [ ]:
from utils.downloader_api import build_client_from_env

cfg, client = build_client_from_env()

print("Base URL:", cfg.base_url)
print("Projects endpoint:", cfg.projects_endpoint)
print("Plots filter endpoint:", cfg.plots_filter_endpoint)
print("Project ID field mapping:", cfg.project_id_field)
print("Project name field mapping:", cfg.project_name_field)

In [ ]:
from utils.filter_search import search_projects
from utils.scraping.paginating_download import download_plots

In [ ]:
# Example 1: search projects (keyword can be project name or project ID text)
projects = await search_projects(client, cfg, keyword="")
print(f"Total projects returned: {len(projects)}")

# Show first few with env-mapped fields
id_field = cfg.project_id_field
name_field = cfg.project_name_field
for row in projects[:5]:
    print({id_field: row.get(id_field), name_field: row.get(name_field)})

In [ ]:
# Example 2: set your project ID and download plots to JSON
PROJECT_ID = 58
OUTPUT_PATH = "./json_downloaded_api/plots/generic_project_58.json"

plots_data, output_file = await download_plots(
    client,
    cfg,
    project_id=PROJECT_ID,
    output_path=OUTPUT_PATH,
    extra_filters={"status": ""},
)

rows_key = cfg.rows_key
print(f"Downloaded {len(plots_data.get(rows_key, []))} records")
print(f"Saved to: {output_file}")

## Notes

- To adapt this notebook for another API, update `.env` only (base URL, endpoints, response keys, field names).
- Reusable API client/runtime helpers are in `utils/downloader_api.py`.
- Reusable search and paginated download wrappers are in `utils/filter_search.py` and `utils/scraping/paginating_download.py`.